In [ ]:
!pip install optuna

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
# load data
import pandas as pd
data=pd.read_csv('/content/drive/MyDrive/data/weighted_data.csv', encoding='cp949', index_col=0)
target=pd.read_csv('/content/drive/MyDrive/data/y_data.csv', encoding='cp949', index_col=0)

In [ ]:
# import models
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier, ExtraTreesClassifier
from xgboost import XGBClassifier

models_li = [RandomForestClassifier(), GradientBoostingClassifier(), ExtraTreesClassifier(), XGBClassifier()]

In [ ]:
# standard_scaling
from sklearn.preprocessing import StandardScaler
ssc = StandardScaler()
data_ssc=ssc.fit_transform(data)

In [ ]:
RandomForestClassifier().get_params()

In [ ]:
import numpy as np
from sklearn.metrics import accuracy_score
from sklearn.model_selection import cross_val_score,KFold
kfold = KFold(n_splits=3,random_state=42,shuffle=True)

In [ ]:
# RandomForestClassifier_parameter_tuning
# based on optuna

import optuna

def rfc_object(trial):
    prms={
    'bootstrap': trial.suggest_categorical('bootstrap',[True, False]),
    'max_depth': trial.suggest_int('max_depth',10,100),
    'min_samples_leaf': trial.suggest_int('min_samples_leaf',1,5),
    'min_samples_split': trial.suggest_int('min_samples_split',2,10),
    'n_estimators': trial.suggest_int('n_estimators',200,2000,step=200)
    }
    model =RandomForestClassifier(**prms)
    result = cross_val_score(
        model,
        data_ssc,
        target['9일 뒤 종가'],
        cv=kfold
    )
    return np.mean(result)

study = optuna.create_study(direction="maximize")
study.optimize(rfc_object,n_trials=20)

In [ ]:
rfc_best_prms=study.best_params
rfc_best_prms

In [ ]:
rfc=RandomForestClassifier(**rfc_best_prms)

In [ ]:
rfc.get_params()

In [ ]:
# GradientBoostingClassifier_parameter_tuning
# based on optuna

import optuna

def gbc_object(trial):
    prms={
    'n_estimators': trial.suggest_int('n_estimators',200,2000,step=200),
    'max_depth': trial.suggest_int('max_depth',10,1000),
    "learning_rate": trial.suggest_float("learning_rate",1e-5,1),
    }
    model =GradientBoostingClassifier(**prms)
    result = cross_val_score(
        model,
        data_ssc,
        target['9일 뒤 종가'],
        cv=kfold
    )
    return np.mean(result)

study = optuna.create_study(direction="maximize")
study.optimize(gbc_object,n_trials=20)

In [ ]:
gbc_best_prms=study.best_params
gbc_best_prms

In [ ]:
gbc=GradientBoostingClassifier(**gbc_best_prms)
gbc.get_params()

In [ ]:
# ExtraTreesClassifier_parameter_tuning
# based on optuna

import optuna

def ettc_object(trial):
    prms={
    'n_estimators': trial.suggest_int('n_estimators',200,2000,step=200),
    'max_depth': trial.suggest_int('max_depth',10,1000),
    }
    model =ExtraTreesClassifier(**prms)
    result = cross_val_score(
        model,
        data_ssc,
        target['9일 뒤 종가'],
        cv=kfold
    )
    return np.mean(result)

study = optuna.create_study(direction="maximize")
study.optimize(ettc_object,n_trials=20)

In [ ]:
ettc_best_prms=study.best_params
ettc_best_prms

In [ ]:
ettc=ExtraTreesClassifier(**ettc_best_prms)
ettc.get_params()

In [ ]:
# XGBClassifier_parameter_tuning
# based on optuna

import optuna

def xgbc_object(trial):
    prms={
    'min_child_weight': trial.suggest_int('min_child_weight',1,10),
    'gamma': trial.suggest_float('gamma',0.5,5),
    "subsample": trial.suggest_float("subsample",1e-1,1),
    'colsample_bytree': trial.suggest_float('colsample_bytree',1e-1,1),
    'max_depth': trial.suggest_int('max_depth', 5, 30)
    }
    model =XGBClassifier(**prms)
    result = cross_val_score(
        model,
        data_ssc,
        target['9일 뒤 종가'],
        cv=kfold
    )
    return np.mean(result)

study = optuna.create_study(direction="maximize")
study.optimize(xgbc_object,n_trials=20)

In [ ]:
xgbc_best_prms=study.best_params
xgbc_best_prms

In [ ]:
xgbc=XGBClassifier(**xgbc_best_prms)
xgbc.get_params()